In [1]:
import pandas as pd
import numpy as np

main = pd.read_pickle("../Data/main_1.df")
main = main.drop(['horse_type_Brown',
           'horse_type_Colt', 'horse_type_Filly', 'horse_type_Gelding',
           'horse_type_Grey', 'horse_type_Horse', 'horse_type_Mare',
           'horse_type_Rig', 'horse_type_Roan', 'horse_country_ARG',
           'horse_country_AUS', 'horse_country_BRZ', 'horse_country_CAN',
           'horse_country_FR', 'horse_country_GB', 'horse_country_GER',
           'horse_country_GR', 'horse_country_IRE', 'horse_country_ITY',
           'horse_country_JPN', 'horse_country_NZ', 'horse_country_SAF',
           'horse_country_SPA', 'horse_country_USA', 'horse_country_ZIM',], axis=1)

In [2]:
# Create a speed figure...
# 1. we must find the average times for each distance
# 2. we must find average times for each venue
# 3. we must find average times for going types
# 3. combine these for a final figure that 'rates' the horse, based on times

In [3]:
dT = []
d = np.unique(main["distance"])
for i in d:
    dT.append(np.mean(main.loc[main["distance"]==i]["finish_time"]))

[57.88047043641483, 70.72472081629347, 83.56744112598697, 96.4732200744387, 101.51651475137706, 110.3856912568306, 124.09736842105262, 138.94263056092842, 149.8474427480916]
[1000 1200 1400 1600 1650 1800 2000 2200 2400]


In [4]:
# create a function that associates a time at a distance to a rating
def rawScore(d, t, D, T):
    # Set the average value to == 90 score
    avgT = T[np.where(D==d)[0][0]]
    score = (t - avgT) * (1000./d)  + 90
    return score

In [5]:
main["raw_score"] = [rawScore(main.iloc[i,:]["distance"], main.iloc[i,:]["finish_time"], d, dT) for i in range(len(main))]

In [6]:
venues = np.unique(main["venue"])
vAvg = []
for v in venues:
    vAvg.append(np.mean(main.loc[main["venue"]==v]["raw_score"]))
vAdjust = [vAvg[i]-90 for i in range(len(vAvg))]

In [7]:
goings = np.unique(main["going"])
gAvg = []
for g in goings:
    gAvg.append(np.mean(main.loc[main["going"]==g]["raw_score"]))
gAdjust = [gAvg[i]-90 for i in range(len(gAvg))]

In [8]:
def refinedScore(venue, going, rawScore, vs, gs, va, ga):
    a = va[list(vs).index(venue)]
    b = ga[list(gs).index(going)]
    return rawScore - a - b
main["refined_score"] = [refinedScore(main.iloc[i,:]["venue"], main.iloc[i,:]["going"], main.iloc[i,:]["raw_score"], venues, goings, vAdjust, gAdjust) for i in range(len(main))]


In [9]:
for i in main.columns:
    print(i)

race_id
horse_no
horse_id
result
won
lengths_behind
horse_age
horse_country
horse_type
horse_rating
horse_gear
declared_weight
actual_weight
draw
position_sec1
position_sec2
position_sec3
position_sec4
position_sec5
position_sec6
behind_sec1
behind_sec2
behind_sec3
behind_sec4
behind_sec5
behind_sec6
time1
time2
time3
time4
time5
time6
finish_time
win_odds
place_odds
trainer_id
jockey_id
race_size
date
venue
race_no
distance
going
config
horse_ratings
prize
race_class
place_combination1
place_combination2
place_combination3
place_combination4
last_race_result
win_percent
avg_distance_time
normal_avg_distance_time
going_type
going_type_record
actual_weight_scaled
declared_weight_scaled
horse_race_count
going_type_fast
going_type_slow
going_type_wet
jockey_record
trainer_record
horse_record
days_since_last_race
weight_change
weight_change_over_time
weight_change_from_average
weight_change_increase
normalized_result
venue_change
venue_record
best_odds
best_win_percent
best_distance_time
b

In [10]:
main["refined_score"]

0        90.586704
1        90.329561
2        90.215276
3        90.372419
4        90.100990
           ...    
79442    89.901113
79443    89.982363
79444    90.069863
79445    90.007363
79446    90.188613
Name: refined_score, Length: 79447, dtype: float64

In [11]:
main.to_pickle("../Data/main_1.df")